In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

# Define the dataset path
data_dir = 'Data'

# Parameters
img_height, img_width = 128, 128  # Resize images to 128x128
batch_size = 32

# Data Augmentation and Preprocessing
data_gen = ImageDataGenerator(
    rescale=1.0/255.0,   # Normalize pixel values
    validation_split=0.2,  # 20% data for validation
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


In [3]:

# Load Training and Validation Data
train_data = data_gen.flow_from_directory(
    directory=data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = data_gen.flow_from_directory(
    directory=data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 1645 images belonging to 9 classes.
Found 407 images belonging to 9 classes.


In [4]:
classes = 9

# Build the Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(classes, activation='softmax')  # 9 output classes
])


In [5]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [6]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5
)

Epoch 1/5
52/52 [==============================] - 75s 1s/step - loss: 1.8615 - accuracy: 0.3222 - val_loss: 1.1600 - val_accuracy: 0.7494
Epoch 2/5
52/52 [==============================] - 49s 950ms/step - loss: 1.0763 - accuracy: 0.6188 - val_loss: 0.8216 - val_accuracy: 0.7076
Epoch 3/5
52/52 [==============================] - 46s 885ms/step - loss: 0.7470 - accuracy: 0.7465 - val_loss: 0.4150 - val_accuracy: 0.8943
Epoch 4/5
52/52 [==============================] - 46s 880ms/step - loss: 0.5318 - accuracy: 0.8237 - val_loss: 0.4984 - val_accuracy: 0.8526
Epoch 5/5
52/52 [==============================] - 44s 838ms/step - loss: 0.4479 - accuracy: 0.8316 - val_loss: 0.2650 - val_accuracy: 0.9189


In [7]:

# Save the Model
# model.save('sign_language_model.h5')

# Evaluate the Model
test_loss, test_accuracy = model.evaluate(val_data)
print(f"Validation Accuracy: {test_accuracy * 100:.2f}%")


13/13 [==============================] - 4s 267ms/step - loss: 0.2903 - accuracy: 0.9263
Validation Accuracy: 92.63%


In [8]:
model.save('sign_language_model_c.h5')

In [11]:
from tensorflow.keras.models import load_model
model = load_model('E:\Downloads\phase 1-sign\phase2\sign_language_model_c.h5')

In [12]:
# Testing
from tensorflow.keras.preprocessing import image

def predict_class(image_path):
    img = image.load_img(image_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    predictions = model.predict(img_array)
    class_idx = np.argmax(predictions)
    class_label = list(train_data.class_indices.keys())[class_idx]
    return class_label

In [14]:

# Test Example
example_image = (r'E:\Downloads\phase 1-sign\phase2\Data\No\Image_1729947685.060301.jpg')
result = predict_class(example_image)
print(f"Predicted Class: {result}")

1/1 [==============================] - 0s 479ms/step
Predicted Class: No


In [15]:

# Test Example
example_image = 'E:\Downloads\phase 1-sign\phase2\Data\Thank You\Image_1729947629.339197.jpg'
result = predict_class(example_image)
print(f"Predicted Class: {result}")

1/1 [==============================] - 0s 49ms/step
Predicted Class: Thank You
